# 🎤 PIPER TTS - TRAINING ITALIANO DEFINITIVO

## 📋 Panoramica
Questo notebook addestra un modello Piper TTS in italiano usando il dataset giacomoarienti/female-LJSpeech-italian.

**Caratteristiche:**
- ✅ Training completo da zero
- ✅ Dataset: 5856 file audio (8h 23m)
- ✅ Voce femminile italiana professionale
- ✅ Sample rate: 16000Hz
- ⏱️ Tempo stimato: 12-16 ore su GPU T4

**Prerequisiti:**
- ✅ Google Colab con GPU (Runtime > Change runtime type > GPU)
- ✅ Dataset scaricato usando `Download_Dataset_Italiano_DEFINITIVO.ipynb`

---

## 🔧 STEP 1: Verifica GPU

In [1]:
# Verifica che la GPU sia disponibile
!nvidia-smi

print("\n" + "="*60)
print("✅ Se vedi info GPU sopra, sei pronto!")
print("❌ Se vedi errore: Runtime > Change runtime type > GPU")
print("="*60)

/bin/bash: line 1: nvidia-smi: command not found

✅ Se vedi info GPU sopra, sei pronto!
❌ Se vedi errore: Runtime > Change runtime type > GPU


## 📂 STEP 2: Monta Google Drive

**IMPORTANTE:** Il dataset deve essere già stato scaricato usando il notebook `Download_Dataset_Italiano_DEFINITIVO.ipynb`

In [2]:
from google.colab import drive

print("📂 Montaggio Google Drive...\n")
drive.mount('/content/drive')
print("\n✅ Google Drive montato con successo!")

📂 Montaggio Google Drive...

Mounted at /content/drive

✅ Google Drive montato con successo!


## 📦 STEP 3: Installazione Piper

In [3]:
print("📦 Installazione Piper Training con Python 3.10...\n")

# Pulisci installazioni precedenti
!rm -rf /content/piper-phonemize /content/piper /content/piper-samples /content/piper-repo /content/miniconda
!pip uninstall -y piper-phonemize piper-train 2>/dev/null || true

# STEP 0: Installa espeak-ng (RICHIESTO da piper_phonemize)
print("="*60)
print("  📦 INSTALLAZIONE ESPEAK-NG")
print("="*60)
print("\n⚠️  piper_phonemize richiede espeak-ng installato nel sistema")
print("💡 Installiamo espeak-ng con apt-get...\n")

!apt-get update -qq
!apt-get install -y espeak-ng

print("✅ espeak-ng installato!\n")

print("="*60)
print("  🐍 INSTALLAZIONE PYTHON 3.10 CON CONDA")
print("="*60)
print("\n⚠️  Colab usa Python 3.12, ma piper_phonemize richiede Python 3.10")
print("💡 Soluzione: installiamo Python 3.10 con conda IN Colab!\n")

# STEP 1: Installa miniconda
print("📥 Download e installazione Miniconda...")
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!chmod +x miniconda.sh
!./miniconda.sh -b -p /content/miniconda -f
!rm miniconda.sh

# Aggiungi conda al PATH
import os
import sys
os.environ['PATH'] = f"/content/miniconda/bin:{os.environ['PATH']}"

print("✅ Miniconda installato!\n")

# STEP 1.5: Accetta i Terms of Service di Conda
print("📝 Accettazione Terms of Service di Conda...")
!/content/miniconda/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!/content/miniconda/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

print("✅ ToS accettati!\n")

# STEP 2: Crea ambiente Python 3.10
print("🔧 Creazione ambiente Python 3.10...")
!/content/miniconda/bin/conda create -n piper python=3.10 -y -q

print("✅ Ambiente Python 3.10 creato!\n")

# STEP 3: Installa piper_phonemize
print("="*60)
print("  📦 INSTALLAZIONE PIPER_PHONEMIZE")
print("="*60)
print("\n💾 Installazione da PyPI...\n")

PYTHON_310 = "/content/miniconda/envs/piper/bin/python"
PIP_310 = "/content/miniconda/envs/piper/bin/pip"

!{PIP_310} install -q piper-phonemize

# Verifica
print("🔍 Verifica piper_phonemize...")
!{PYTHON_310} -c "import piper_phonemize; print('✅ piper_phonemize importato correttamente!')"

print("\n" + "="*60)
print("  📦 INSTALLAZIONE DIPENDENZE TRAINING")
print("="*60 + "\n")

# IMPORTANTE: Installa versioni compatibili specifiche
print("📦 Installazione dipendenze con versioni compatibili...")

# Downgrade pip per compatibilità
!{PIP_310} install -q "pip<24.1"

# Installa numpy 1.x (richiesto da pytorch-lightning)
!{PIP_310} install -q "numpy<2.0"

# Installa torch compatibile con pytorch-lightning e CUDA 11.7
!{PIP_310} install -q torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# Installa pytorch-lightning 1.8 con dipendenze compatibili
# NOTA: pytorch-lightning 1.7.7 ha requisiti invalidi (torch >=1.9.*) incompatibili con pip 24.1+
# Usiamo 1.8.6 che è compatibile e non ha problemi di requisiti
!{PIP_310} install -q "pytorch-lightning==1.8.6" "torchmetrics==0.11.4"

# Altre dipendenze
!{PIP_310} install -q onnxruntime
!{PIP_310} install -q "librosa<1.0"
!{PIP_310} install -q soundfile
!{PIP_310} install -q tensorboard
!{PIP_310} install -q pandas tqdm

print("✅ Dipendenze installate!\n")

# STEP 4: Clona repository Piper per gli script di training
print("="*60)
print("  📥 DOWNLOAD REPOSITORY PIPER")
print("="*60 + "\n")

!git clone -q https://github.com/rhasspy/piper.git /content/piper-repo

print("✅ Repository clonato!\n")

# STEP 5: Installa piper_train
print("📦 Installazione piper_train...")
%cd /content/piper-repo/src/python
!{PIP_310} install -q -e .
%cd /content

print("✅ piper_train installato!\n")

# VERIFICA FINALE
print("="*60)
print("  ✅ VERIFICA INSTALLAZIONE")
print("="*60 + "\n")

print("🐍 Versione Python:")
!{PYTHON_310} --version

print("\n📦 Moduli installati:")
!{PYTHON_310} -c "import piper_phonemize; print('  ✓ piper_phonemize')"
!{PYTHON_310} -c "import numpy; print(f'  ✓ numpy {{numpy.__version__}}')"
!{PYTHON_310} -c "import torch; print(f'  ✓ torch {{torch.__version__}}')"
!{PYTHON_310} -c "import pytorch_lightning; print(f'  ✓ pytorch_lightning {{pytorch_lightning.__version__}}')"
!{PYTHON_310} -c "import piper_train; print('  ✓ piper_train')"

print("\n✅ INSTALLAZIONE COMPLETATA!")
print("\n💡 IMPORTANTE: Tutti i comandi Python useranno:")
print(f"   {PYTHON_310}")
print("\n" + "="*60)

📦 Installazione Piper Training con Python 3.10...

  📦 INSTALLAZIONE ESPEAK-NG

⚠️  piper_phonemize richiede espeak-ng installato nel sistema
💡 Installiamo espeak-ng con apt-get...

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 43 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd6

## 📁 STEP 4: Verifica Dataset

**IMPORTANTE:** Prima di eseguire questa cella, assicurati di aver scaricato il dataset usando il notebook `Download_Dataset_Italiano_DEFINITIVO.ipynb`

In [4]:
import os
import pandas as pd
from pathlib import Path

# Path dataset (scaricato dal notebook di download)
DATASET_DIR = "/content/drive/MyDrive/ljspeech_italian"
OUTPUT_DIR = "/content/piper_output"

print("="*60)
print("  🔍 VERIFICA DATASET")
print("="*60)

# Verifica esistenza directory
if not os.path.exists(DATASET_DIR):
    print("\n❌ ERRORE: Dataset non trovato!")
    print(f"   Path cercato: {DATASET_DIR} (Google Drive)")
    print("\n💡 SOLUZIONE:")
    print("   1. Esegui prima il notebook 'Download_Dataset_Italiano_DEFINITIVO.ipynb'")
    print("   2. Assicurati che il dataset sia in Google Drive: /content/drive/MyDrive/ljspeech_italian/")
    raise FileNotFoundError(f"Dataset non trovato in {DATASET_DIR}")

# Verifica metadata.csv
metadata_path = f"{DATASET_DIR}/metadata.csv"
if not os.path.exists(metadata_path):
    raise FileNotFoundError("metadata.csv non trovato!")

# Verifica wavs/
wavs_dir = f"{DATASET_DIR}/wavs"
if not os.path.exists(wavs_dir):
    raise FileNotFoundError("Cartella wavs/ non trovata!")

# Carica e verifica metadata
metadata = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
num_metadata = len(metadata)

# Conta file WAV
wav_files = list(Path(wavs_dir).glob("*.wav"))
num_wavs = len(wav_files)

print(f"\n✅ Dataset trovato!")
print(f"   📁 Path: {DATASET_DIR}")
print(f"   📄 Trascrizioni: {num_metadata}")
print(f"   🎵 File audio: {num_wavs}")

if num_metadata == num_wavs:
    print(f"   ✅ Corrispondenza perfetta!")
else:
    print(f"   ⚠️  Warning: {num_metadata} trascrizioni vs {num_wavs} file audio")

# Crea directory output
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"\n✅ Directory output creata: {OUTPUT_DIR}")

print("\n" + "="*60)

  🔍 VERIFICA DATASET

✅ Dataset trovato!
   📁 Path: /content/drive/MyDrive/ljspeech_italian
   📄 Trascrizioni: 5856
   🎵 File audio: 5856
   ✅ Corrispondenza perfetta!

✅ Directory output creata: /content/piper_output



## 🎛️ STEP 5: Configurazione Training

Qui configuriamo tutti i parametri del modello e del training.

In [5]:
import json

print("="*60)
print("  ⚙️ CONFIGURAZIONE TRAINING")
print("="*60)

# Configurazione completa
config = {
    "audio": {
        "sample_rate": 16000,
        "max_wav_value": 32767.0,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024
    },
    "model": {
        "name": "vits",
        "hidden_channels": 192,
        "inter_channels": 192,
        "filter_channels": 768,
        "n_heads": 2,
        "n_layers": 6,
        "kernel_size": 3,
        "p_dropout": 0.1
    },
    "training": {
        "epochs": 10000,
        "learning_rate": 0.0002,
        "batch_size": 16,
        "log_interval": 100,
        "save_interval": 1000,
        "num_workers": 4
    },
    "dataset": {
        "path": DATASET_DIR,
        "text_cleaners": ["basic_cleaners"],
        "language": "it-it"
    }
}

# Salva configurazione
config_path = f"{OUTPUT_DIR}/config.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Configurazione salvata: {config_path}")
print(f"\n📝 Parametri principali:")
print(f"   • Sample rate: {config['audio']['sample_rate']} Hz")
print(f"   • Lingua: {config['dataset']['language']} (Italiano)")
print(f"   • Epochs: {config['training']['epochs']:,}")
print(f"   • Batch size: {config['training']['batch_size']}")
print(f"   • Learning rate: {config['training']['learning_rate']}")
print(f"   • Salvataggio checkpoint ogni: {config['training']['save_interval']} steps")

print("\n" + "="*60)

  ⚙️ CONFIGURAZIONE TRAINING

✅ Configurazione salvata: /content/piper_output/config.json

📝 Parametri principali:
   • Sample rate: 16000 Hz
   • Lingua: it-it (Italiano)
   • Epochs: 10,000
   • Batch size: 16
   • Learning rate: 0.0002
   • Salvataggio checkpoint ogni: 1000 steps



## 🔄 STEP 6: Preprocessing Dataset

Prepara il dataset per il training (estrazione features, phonemization, etc.)

In [9]:
print("="*60)
print("  🚀 PREPROCESSING COMPLETO - 5856 FILE")
print("="*60)
print("\n⏱️  Tempo stimato: 25-35 minuti con 1 worker\n")

import os

# Percorsi
DATASET_DIR = "/content/drive/MyDrive/ljspeech_italian"
OUTPUT_DIR = "/content/drive/MyDrive/piper_training/output"
PREPROCESSED_DIR = f"{OUTPUT_DIR}/preprocessed_full"

os.makedirs(PREPROCESSED_DIR, exist_ok=True)

print(f"📁 Dataset: {DATASET_DIR}")
print(f"📁 Output: {PREPROCESSED_DIR}\n")

# Python 3.10
PYTHON_310 = "/content/miniconda/envs/piper/bin/python"

if not os.path.exists(PYTHON_310):
    print("❌ Python 3.10 non trovato - esegui prima la cella 3")
    raise FileNotFoundError("Python 3.10 non installato")

# Configura eSpeak-ng
print("🔧 Configurazione eSpeak-ng...")
!{PYTHON_310} -c "from piper_phonemize import phonemize_espeak; phonemize_espeak('test', 'it')"
print("✅ eSpeak-ng configurato\n")

# Vai nella directory piper per il preprocessing
%cd /content/piper-repo/src/python

print("🚀 Avvio preprocessing di 5856 file...")
print("💡 Questo richiederà 25-35 minuti - NON interrompere!\n")

# PREPROCESSING COMPLETO
!{PYTHON_310} -m piper_train.preprocess \
    --input-dir {DATASET_DIR} \
    --output-dir {PREPROCESSED_DIR} \
    --language it \
    --sample-rate 16000 \
    --dataset-format ljspeech \
    --single-speaker \
    --max-workers 1

%cd /content

print("\n" + "="*60)
print("✅ PREPROCESSING COMPLETATO!")
print("="*60)

# Verifica finale
cache_dir = f"{PREPROCESSED_DIR}/cache/16000"
if os.path.exists(cache_dir):
    pt_files = [f for f in os.listdir(cache_dir) if f.endswith('.pt') and not f.endswith('.spec.pt')]
    spec_files = [f for f in os.listdir(cache_dir) if f.endswith('.spec.pt')]

    print(f"\n📊 Risultato finale:")
    print(f"   Audio processati (.pt): {len(pt_files)}")
    print(f"   Spettrogrammi (.spec.pt): {len(spec_files)}")
    print(f"   Dataset originale: 5856 file")

    if len(pt_files) >= 5800:
        print(f"\n🎉 SUCCESSO! Dataset preprocessato completamente!")
        print(f"\n📁 File pronti in:")
        print(f"   {PREPROCESSED_DIR}")
        print(f"\n✅ Puoi procedere con il TRAINING (prossima cella)!")
    else:
        print(f"\n⚠️  Attenzione: solo {len(pt_files)} file processati")
        print(f"   Verifica se ci sono stati errori sopra")
else:
    print("\n❌ Cache directory non trovata - controlla errori sopra")

  🚀 PREPROCESSING COMPLETO - 5856 FILE

⏱️  Tempo stimato: 25-35 minuti con 1 worker

📁 Dataset: /content/drive/MyDrive/ljspeech_italian
📁 Output: /content/drive/MyDrive/piper_training/output/preprocessed_full

🔧 Configurazione eSpeak-ng...
✅ eSpeak-ng configurato

/content/piper-repo/src/python
🚀 Avvio preprocessing di 5856 file...
💡 Questo richiederà 25-35 minuti - NON interrompere!

INFO:preprocess:Single speaker dataset
INFO:preprocess:Wrote dataset config
INFO:preprocess:Processing 5856 utterance(s) with 1 worker(s)
/content

✅ PREPROCESSING COMPLETATO!

📊 Risultato finale:
   Audio processati (.pt): 5856
   Spettrogrammi (.spec.pt): 5856
   Dataset originale: 5856 file

🎉 SUCCESSO! Dataset preprocessato completamente!

📁 File pronti in:
   /content/drive/MyDrive/piper_training/output/preprocessed_full

✅ Puoi procedere con il TRAINING (prossima cella)!


## 🚀 STEP 7: AVVIO TRAINING

**⚠️ ATTENZIONE:** Il training richiederà **12-16 ore** su GPU T4.

**Consigli:**
- 💡 Tieni aperta la tab del browser per evitare disconnessioni
- 💡 Il training salva checkpoint ogni 1000 step, quindi puoi riprenderlo se si interrompe
- 💡 Puoi monitorare i log per vedere i progressi

In [ ]:
print("="*60)
print("  🎯 AVVIO TRAINING")
print("="*60)
print("\n⏱️  Tempo stimato: 12-16 ore")
print("💾 Checkpoint salvati ogni 1000 step")
print("📊 Log ogni 100 step\n")
print("-"*60)

CHECKPOINT_DIR = f"{OUTPUT_DIR}/checkpoints"

# Path Python 3.10 (installato nello STEP 3)
PYTHON_310 = "/content/miniconda/envs/piper/bin/python"

# Verifica che Python 3.10 sia installato
import os
if not os.path.exists(PYTHON_310):
    print("❌ ERRORE: Python 3.10 non trovato!")
    print("\n💡 SOLUZIONE:")
    print("   1. Torna allo STEP 3")
    print("   2. Esegui la cella di installazione")
    print("   3. Aspetta che finisca (~5 minuti)")
    print("   4. Poi torna qui e riprova")
    raise FileNotFoundError(f"Python 3.10 non installato. Path cercato: {PYTHON_310}")

%cd /content/piper-repo/src/python

!{PYTHON_310} -m piper_train \
    --dataset-dir {PREPROCESSED_DIR} \
    --output-dir {CHECKPOINT_DIR} \
    --config {config_path} \
    --restore-checkpoint

%cd /content

print("\n" + "="*60)
print("🎉 TRAINING COMPLETATO!")
print("="*60)

## 📊 STEP 8: Verifica Checkpoint

Visualizza i checkpoint salvati durante il training.

In [ ]:
import glob

print("="*60)
print("  📊 CHECKPOINT SALVATI")
print("="*60)

checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/*.pt"))

if len(checkpoints) == 0:
    print("\n❌ Nessun checkpoint trovato!")
else:
    print(f"\n✅ Trovati {len(checkpoints)} checkpoint:\n")

    for ckpt in checkpoints[-10:]:  # Mostra ultimi 10
        size_mb = os.path.getsize(ckpt) / (1024*1024)
        print(f"   📁 {os.path.basename(ckpt)} ({size_mb:.1f} MB)")

    if len(checkpoints) > 10:
        print(f"\n   ... e altri {len(checkpoints)-10} checkpoint")

print("\n" + "="*60)

## 🎵 STEP 9: Export Modello ONNX

Converte il checkpoint PyTorch in formato ONNX (richiesto da Piper per l'inferenza).

In [ ]:
import glob
import os

print("="*60)
print("  📦 EXPORT MODELLO ONNX")
print("="*60)

# Path Python 3.10 (installato nello STEP 3)
PYTHON_310 = "/content/miniconda/envs/piper/bin/python"

# Verifica che Python 3.10 sia installato
if not os.path.exists(PYTHON_310):
    print("\n❌ ERRORE: Python 3.10 non trovato!")
    print("\n💡 SOLUZIONE:")
    print("   1. Torna allo STEP 3")
    print("   2. Esegui la cella di installazione")
    print("   3. Aspetta che finisca (~5 minuti)")
    print("   4. Poi torna qui e riprova")
    raise FileNotFoundError(f"Python 3.10 non installato. Path cercato: {PYTHON_310}")

# Trova ultimo checkpoint
checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/*.pt"))

if len(checkpoints) == 0:
    print("\n❌ Nessun checkpoint trovato!")
    print("Assicurati che il training sia completato.")
else:
    latest_checkpoint = checkpoints[-1]
    print(f"\n📁 Usando checkpoint: {os.path.basename(latest_checkpoint)}")

    model_output = f"{OUTPUT_DIR}/model.onnx"

    %cd /content/piper-repo/src/python

    !{PYTHON_310} -m piper_train.export_onnx \
        {latest_checkpoint} \
        {model_output}

    %cd /content

    if os.path.exists(model_output):
        size_mb = os.path.getsize(model_output) / (1024*1024)
        print(f"\n✅ Modello esportato con successo!")
        print(f"   📁 Path: {model_output}")
        print(f"   💾 Dimensione: {size_mb:.1f} MB")
    else:
        print("\n❌ Errore durante l'export!")

print("\n" + "="*60)

## 🧪 STEP 10: Test Modello

Prova il modello generando audio da testo.

In [ ]:
from IPython.display import Audio, display
import subprocess
import os

print("="*60)
print("  🧪 TEST MODELLO")
print("="*60)

# Testi di test in italiano
test_texts = [
    "Buongiorno, questo è un test del modello Piper addestrato in italiano.",
    "La sintesi vocale funziona correttamente con frasi più lunghe e complesse.",
    "I Malavoglia è un romanzo di Giovanni Verga."
]

model_path = f"{OUTPUT_DIR}/model.onnx"

if not os.path.exists(model_path):
    print("\n❌ Modello non trovato! Esegui prima lo STEP 8.")
else:
    print("\n🎤 Generazione audio di test...\n")

    for i, text in enumerate(test_texts, 1):
        output_wav = f"{OUTPUT_DIR}/test_{i}.wav"

        print(f"{i}. Testo: \"{text}\"")

        # Genera audio con Piper
        result = subprocess.run(
            ["piper", "--model", model_path, "--output_file", output_wav],
            input=text.encode('utf-8'),
            capture_output=True
        )

        if os.path.exists(output_wav):
            print(f"   ✅ Audio generato: {output_wav}")
            display(Audio(output_wav))
            print()
        else:
            print(f"   ❌ Errore nella generazione")
            if result.stderr:
                print(f"   Errore: {result.stderr.decode('utf-8')}")

print("="*60)

## 💾 STEP 11: Download Modello

Scarica il modello finale sul tuo computer.

In [ ]:
import shutil
from google.colab import files

print("="*60)
print("  💾 DOWNLOAD MODELLO")
print("="*60)

model_path = f"{OUTPUT_DIR}/model.onnx"

if not os.path.exists(model_path):
    print("\n❌ Modello non trovato!")
else:
    print("\n📦 Creazione archivio per il download...\n")

    # Crea directory per il package
    package_dir = f"{OUTPUT_DIR}/piper_model_italiano"
    os.makedirs(package_dir, exist_ok=True)

    # Copia file necessari
    shutil.copy(model_path, package_dir)
    shutil.copy(config_path, package_dir)

    # Crea README
    readme_content = f"""# Piper TTS - Modello Italiano

## Informazioni Modello
- **Lingua**: Italiano (it-it)
- **Voce**: Femminile
- **Dataset**: giacomoarienti/female-LJSpeech-italian
- **Sample Rate**: 16000 Hz
- **Ore di training**: ~8h 23m di audio

## Utilizzo
```bash
echo "Ciao, questo è un test" | piper --model model.onnx --output_file output.wav
```

## File Inclusi
- `model.onnx`: Modello ONNX per l'inferenza
- `config.json`: Configurazione del modello
"""

    with open(f"{package_dir}/README.md", 'w', encoding='utf-8') as f:
        f.write(readme_content)

    # Crea ZIP
    archive_path = shutil.make_archive(
        f"{OUTPUT_DIR}/piper_model_italiano",
        'zip',
        package_dir
    )

    archive_size_mb = os.path.getsize(archive_path) / (1024*1024)

    print(f"✅ Archivio creato:")
    print(f"   📁 {os.path.basename(archive_path)}")
    print(f"   💾 Dimensione: {archive_size_mb:.1f} MB")
    print(f"\n📥 Download in corso...\n")

    # Download
    files.download(archive_path)

    print("\n✅ Download completato!")

print("\n" + "="*60)

## 🎉 COMPLETATO!

### Risultati:
- ✅ Modello Piper TTS italiano addestrato
- ✅ File ONNX pronto per l'uso
- ✅ Configurazione salvata

### Prossimi Passi:
1. **Testa il modello** con frasi diverse
2. **Fine-tuning** se necessario con più dati
3. **Integra** in applicazioni usando Piper CLI

### Utilizzo del Modello:
```bash
# Sintesi da testo
echo "Buongiorno, come stai?" | piper --model model.onnx --output_file output.wav

# Da file di testo
cat testo.txt | piper --model model.onnx --output_file output.wav
```

### Troubleshooting:
- **Qualità audio bassa**: Aumenta epochs o migliora dataset
- **OOM Error**: Riduci batch_size nella configurazione
- **Training interrotto**: Riavvia dalla cella STEP 6 (riprenderà dall'ultimo checkpoint)

### Risorse:
- [Piper GitHub](https://github.com/rhasspy/piper)
- [Piper Documentation](https://github.com/rhasspy/piper/blob/master/README.md)

---

**Buon TTS! 🎤🚀**